In [1]:
import sklearn
sklearn.show_versions()


System:
    python: 3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]
executable: c:\programdata\miniconda3\python.exe
   machine: Windows-10-10.0.18362-SP0

Python dependencies:
          pip: 20.1.1
   setuptools: 40.2.0
      sklearn: 0.23.1
        numpy: 1.16.0
        scipy: 1.2.0
       Cython: 0.29.16
       pandas: 0.24.0
   matplotlib: 3.0.2
       joblib: 0.14.1
threadpoolctl: 2.1.0

Built with OpenMP: True


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import explained_variance_score, max_error
from sklearn.metrics import mean_squared_log_error, median_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import pickle

In [3]:
train_data= pd.read_csv('PAM_train.csv')
test_data=pd.read_csv('PAM_test.csv')

In [4]:
test_data.shape

(13096, 29)

In [5]:
col_drop = ['UnitNumber', 'Cycle','Failure', 'Op_Setting_3', 'Sensor_1', 'Sensor_5', 
            'Sensor_10', 'Sensor_16', 'Sensor_18', 'Sensor_19'] 

In [6]:
train_data.columns

Index(['UnitNumber', 'Cycle', 'Op_Setting_1', 'Op_Setting_2', 'Op_Setting_3',
       'Sensor_1', 'Sensor_2', 'Sensor_3', 'Sensor_4', 'Sensor_5', 'Sensor_6',
       'Sensor_7', 'Sensor_8', 'Sensor_9', 'Sensor_10', 'Sensor_11',
       'Sensor_12', 'Sensor_13', 'Sensor_14', 'Sensor_15', 'Sensor_16',
       'Sensor_17', 'Sensor_18', 'Sensor_19', 'Sensor_20', 'Sensor_21',
       'Target_RUL', 'Failure', 'Cycle_norm'],
      dtype='object')

In [7]:
def process_cols(df):
    t_df = df.drop(col_drop, axis = 1)
    print(t_df.shape)
    # set up features and target variable 
    t_Y = t_df['Target_RUL']
    t_X = t_df.drop(['Target_RUL'], axis = 1)
    
    return t_X, t_Y

In [8]:
X, Y = process_cols(train_data)
#test_X, test_Y = process_cols(test_data)

(20631, 19)


In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.15, random_state=1234)

In [10]:
def createOutParam(modelName, testX, testY, predY):
    mse = mean_squared_error(testY, predY)
    mae = mean_absolute_error(testY, predY)
    rsquared = r2_score(testY, predY)
    explained_variance =  explained_variance_score(testY, predY)
    max_err = max_error(testY, predY)
    rmse = np.sqrt(mean_squared_error(testY, predY))
   # msle = mean_squared_log_error(testY, predY)
    medianae = median_absolute_error(testY, predY)
    model_perf = {'ModelName': modelName,
                  'explained_variance' : explained_variance,
                  'mean_absolute_error': mae,
                  'median_absolute_error':medianae,
                  'max_error':max_err,
                  'R_squared': rsquared,
                  'mean_squared_error': mse,
                  'root_mean_squared_error':rmse,
                 # 'mean_squared_log_error':msle
                 }        
    return model_perf

In [11]:
model_perf = []

## Model # 1 : RandomForestRegressor

In [12]:
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
rf = ensemble.RandomForestRegressor()
# set up 10-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(10)
# pipeline standardization and model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', rf) ])
# tune the model
my_min_samples_leaf = [2, 10, 25, 50, 100]
my_max_depth = [7, 8, 9, 10, 11, 12]
# run the model using gridsearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_rf = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__min_samples_leaf = my_min_samples_leaf, model__max_depth = my_max_depth)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )
optimized_rf.fit(train_X, train_Y)
# show the best model estimators
print(optimized_rf.best_estimator_)
# evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = optimized_rf.predict(test_X)
print("Random Forest Mean Squared Error: ", mean_squared_error(test_Y, y_pred))
print("Random Forest Mean Absolute Error: ", mean_absolute_error(test_Y, y_pred))
print("Random Forest r-squared: ", r2_score(test_Y, y_pred))

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 21.0min finished


Pipeline(steps=[('standardize', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=12, min_samples_leaf=10))])
Random Forest Mean Squared Error:  1286.8095958034892
Random Forest Mean Absolute Error:  25.285723228010845
Random Forest r-squared:  0.7252980965958917


In [13]:
y_pred_rf = optimized_rf.predict(test_X)
model_perf_rf = createOutParam('RandomForestRegressor', test_X, test_Y, y_pred_rf)

In [14]:
model_perf.append(model_perf_rf)
model_perf_rf

{'ModelName': 'RandomForestRegressor',
 'explained_variance': 0.7253156492712516,
 'mean_absolute_error': 25.285723228010845,
 'median_absolute_error': 17.499859903110888,
 'max_error': 156.99759740052946,
 'R_squared': 0.7252980965958917,
 'mean_squared_error': 1286.8095958034892,
 'root_mean_squared_error': 35.87212839801242}

## Model # 2 : Elastic Net

In [15]:
# Elastic Net GLM
# create holdout

from sklearn.linear_model import ElasticNet
glm_net = ElasticNet()

# set up 10-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(10)
# pipeline standardization and model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', glm_net) ])
# tune the model
my_alpha = np.linspace(.01, 1, num=5)
my_l1_ratio = np.linspace(.01, 1, num=3)
# run the model using gridsearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_glm_net = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__l1_ratio = my_l1_ratio, model__alpha = my_alpha)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )
optimized_glm_net.fit(train_X, train_Y)
# show the best model estimators
print(optimized_glm_net.best_estimator_)
y_pred_net = optimized_glm_net.predict(test_X)
print("GLM Elastic Net Mean Squared Error: ", mean_squared_error(test_Y, y_pred))
print("GLM Elastic Net Mean Absolute Error: ", mean_absolute_error(test_Y, y_pred))
print("GLM Elastic Net r-squared: ", r2_score(test_Y, y_pred))

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    9.1s finished


Pipeline(steps=[('standardize', StandardScaler()),
                ('model', ElasticNet(alpha=0.01, l1_ratio=0.505))])
GLM Elastic Net Mean Squared Error:  1286.8095958034892
GLM Elastic Net Mean Absolute Error:  25.285723228010845
GLM Elastic Net r-squared:  0.7252980965958917


In [16]:
y_pred_elnet = optimized_glm_net.predict(test_X)
model_perf_glm_net = createOutParam('ElasticNet', test_X, test_Y, y_pred_elnet)


In [17]:
model_perf.append(model_perf_glm_net)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654}]

## Model # 3 : Support Vector Machine Regressor

In [18]:
from sklearn import svm
from sklearn.svm import SVR
svm = svm.SVR()
# set up 10-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(10)
# pipeline standardization and model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', svm) ])
# tune the model
my_C = [1]
my_epsilon = [.05, .1, .15]
# run the model using gridsearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_svm = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__C = my_C, model__epsilon = my_epsilon)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )
optimized_svm.fit(train_X, train_Y)
# show the best model estimators
print(optimized_svm.best_estimator_)
# evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred_svm = optimized_svm.predict(test_X)
#print("SVM Mean Squared Error: ", mean_squared_error(test_Y, y_pred))
#print("SVM Mean Absolute Error: ", mean_absolute_error(test_Y, y_pred))
#print("SVM r-squared: ", r2_score(test_Y, y_pred))

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.4min finished


Pipeline(steps=[('standardize', StandardScaler()),
                ('model', SVR(C=1, epsilon=0.15))])


In [19]:
y_pred_svm = optimized_svm.predict(test_X)
model_perf_svm = createOutParam('SVM_Regressor', test_X, test_Y, y_pred_svm)
model_perf.append(model_perf_svm)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215}]

## Model # 4 Gradient Boosting Regressor

In [20]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
# set up 5-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(5)
# pipeline standardization and model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', gb) ])
# tune the model

my_learning_rate = [0.05, 0.005, 0.5, 0.0085]
my_n_estimators= [100, 200, 500]
my_max_depth = [7]
my_alpha = [0.5, 0.9]

# run the model using gridsearch, select the model with best search
from sklearn.model_selection import GridSearchCV


optimized_gb = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__max_depth = my_max_depth, model__n_estimators = my_n_estimators,
                                              model__learning_rate = my_learning_rate, model__alpha = my_alpha, )
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 10
                            , n_jobs = -1
                           )
optimized_gb.fit(train_X, train_Y)
# show the best model estimators
print(optimized_gb.best_estimator_)
# evaluate metrics on holdout

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 17.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 21.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 35.2min finished


Pipeline(steps=[('standardize', StandardScaler()),
                ('model',
                 GradientBoostingRegressor(alpha=0.5, learning_rate=0.0085,
                                           max_depth=7, n_estimators=500))])


In [21]:
y_pred_gb = optimized_gb.predict(test_X)
model_perf_gb = createOutParam('GradientBoostingRegressor', test_X, test_Y, y_pred_gb)
model_perf_gb

{'ModelName': 'GradientBoostingRegressor',
 'explained_variance': 0.7201368696726207,
 'mean_absolute_error': 25.611803551210578,
 'median_absolute_error': 17.357777577332314,
 'max_error': 161.784601286633,
 'R_squared': 0.7201205705296294,
 'mean_squared_error': 1311.0631235076198,
 'root_mean_squared_error': 36.20860565539109}

In [22]:
model_perf.append(model_perf_gb)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215},
 

## Model 5 - Voting Regressor

In [23]:
from sklearn.ensemble import VotingRegressor
vr = VotingRegressor([('svm', optimized_svm), ('glm', optimized_glm_net),
                      ('rf', optimized_rf), ('gb', optimized_gb)])
vr.fit(train_X, train_Y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.1min finished


Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    9.5s finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 16.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 26.1min finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 27.4min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 32.6min finished


VotingRegressor(estimators=[('svm',
                             GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
                                          estimator=Pipeline(steps=[('standardize',
                                                                     StandardScaler()),
                                                                    ('model',
                                                                     SVR())]),
                                          n_jobs=-1,
                                          param_grid={'model__C': [1],
                                                      'model__epsilon': [0.05,
                                                                         0.1,
                                                                         0.15]},
                                          scoring='neg_mean_squared_error',
                                          verbose=1)),
                            ('glm',
       

In [24]:
y_pred_vr = vr.predict(test_X)
model_perf_vr = createOutParam('VotingRegressor', test_X, test_Y, y_pred_vr)
model_perf.append(model_perf_vr)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215},
 

## Model 6 - AdaBoost Regressor

In [25]:
from sklearn.ensemble import AdaBoostRegressor
ad = ensemble.AdaBoostRegressor()
# set up 5-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(5)
# pipeline standardization and model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', ad) ])
# tune the model

my_learning_rate = [0.05, 0.005, 0.5]
my_n_estimators= [50, 100]


# run the model using gridsearch, select the model with best search
from sklearn.model_selection import GridSearchCV


optimized_ad = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__n_estimators = my_n_estimators,
                                              model__learning_rate = my_learning_rate)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 10
                            , n_jobs = -1
                           )
optimized_ad.fit(train_X, train_Y)
# show the best model estimators
print(optimized_ad.best_estimator_)
# evaluate metrics on holdout

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  1.2min remaining:    7.7s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.3min finished


Pipeline(steps=[('standardize', StandardScaler()),
                ('model', AdaBoostRegressor(learning_rate=0.05))])


In [26]:
y_pred_ad = optimized_ad.predict(test_X)
model_perf_ad = createOutParam('AdaBoostRegressor', test_X, test_Y, y_pred_ad)
model_perf.append(model_perf_ad)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215},
 

## Model 7 - HistGradientBoostingRegressor

In [27]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
hgb = HistGradientBoostingRegressor(learning_rate=0.05)

hgb.fit(train_X, train_Y)


HistGradientBoostingRegressor(learning_rate=0.05)

In [28]:
y_pred_hgb = hgb.predict(test_X)
model_perf_hgb = createOutParam('HistGradientBoostingRegressor', test_X, test_Y, y_pred_hgb)
model_perf.append(model_perf_hgb)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215},
 

In [29]:
## Model - XGBoost
import xgboost as xgb
#data_dmatrix = xgb.DMatrix(data=X,label=Y)
#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 500)
xg_reg = xgb.XGBRegressor(learning_rate = 0.1,  n_estimators = 300)

xg_reg.fit(train_X,train_Y)

c:\programdata\miniconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(n_estimators=300)

In [30]:
y_pred_xgb = xg_reg.predict(test_X)
model_perf_xgb = createOutParam('XGBoost', test_X, test_Y, y_pred_xgb)
model_perf_xgb

{'ModelName': 'XGBoost',
 'explained_variance': 0.7213124170589515,
 'mean_absolute_error': 25.61287148726776,
 'median_absolute_error': 17.725902557373047,
 'max_error': 163.44125366210938,
 'R_squared': 0.7213108167624116,
 'mean_squared_error': 1305.4875513884128,
 'root_mean_squared_error': 36.13153126271308}

In [31]:
model_perf.append(model_perf_xgb)

## Model 8 - StackingRegressor

In [40]:
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
#estimators = [('hgb', hgb), ('gb',optimized_gb), ('rf',optimized_rf), ('krr',KRR)]
from sklearn.linear_model import Lasso
#KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

estimators = [('gb',optimized_gb), ('rf',optimized_rf), ('xgb', xg_reg)]
stacking_regressor = StackingRegressor(
    estimators=estimators, 
    final_estimator=lasso
)
stacking_regressor.fit(train_X, train_Y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 27.8min finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | el

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 16.9min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 19.6min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 25.5min finished
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 11.1min

Fitting 5 folds for each of 24 candidates, totalling 120 fits


...

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.8min finished


Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.8min finished


StackingRegressor(estimators=[('gb',
                               GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                                            estimator=Pipeline(steps=[('standardize',
                                                                       StandardScaler()),
                                                                      ('model',
                                                                       GradientBoostingRegressor())]),
                                            n_jobs=-1,
                                            param_grid={'model__alpha': [0.5,
                                                                         0.9],
                                                        'model__learning_rate': [0.05,
                                                                                 0.005,
                                                                                 0.5,
                               

In [41]:
y_pred_str = stacking_regressor.predict(test_X)
model_perf_str = createOutParam('Stacking_Regressor', test_X, test_Y, y_pred_str)
#model_perf.append(model_perf_str)
model_perf_str

{'ModelName': 'Stacking_Regressor',
 'explained_variance': 0.7259018811077935,
 'mean_absolute_error': 25.259121823504188,
 'median_absolute_error': 17.293106523973634,
 'max_error': 160.9886546342101,
 'R_squared': 0.7258953337758397,
 'mean_squared_error': 1284.011906655347,
 'root_mean_squared_error': 35.83311187512671}

In [42]:

model_perf.append(model_perf_str)
model_perf

[{'ModelName': 'RandomForestRegressor',
  'explained_variance': 0.7253156492712516,
  'mean_absolute_error': 25.285723228010845,
  'median_absolute_error': 17.499859903110888,
  'max_error': 156.99759740052946,
  'R_squared': 0.7252980965958917,
  'mean_squared_error': 1286.8095958034892,
  'root_mean_squared_error': 35.87212839801242},
 {'ModelName': 'ElasticNet',
  'explained_variance': 0.6484865452146342,
  'mean_absolute_error': 31.158948268857973,
  'median_absolute_error': 25.982292588273367,
  'max_error': 175.4652973870921,
  'R_squared': 0.6484864344961769,
  'mean_squared_error': 1646.6250271298743,
  'root_mean_squared_error': 40.578627713734654},
 {'ModelName': 'SVM_Regressor',
  'explained_variance': 0.6887975104851101,
  'mean_absolute_error': 26.834332579995383,
  'median_absolute_error': 17.682102178812983,
  'max_error': 182.32421107266794,
  'R_squared': 0.6791436811906638,
  'mean_squared_error': 1503.0146671778123,
  'root_mean_squared_error': 38.768733112881215},
 

In [33]:
import pickle
def save_model(model, filename):
    pkl_filename = filename 
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)
    
    return


In [43]:
save_model(stacking_regressor, 'Model/StackingRegressor.pkl')

In [34]:
save_model(optimized_gb, 'Model/GradientBoost.pkl')

In [35]:
save_model(xg_reg, 'Model/XGBoost.pkl')

In [36]:
save_model(optimized_rf, 'Model/RandomForest.pkl')



In [37]:
save_model(optimized_glm_net, 'Model/ElasticNet.pkl')
save_model(optimized_svm, 'Model/SVMRegressor.pkl')
save_model(vr, 'Model/VotingRegressor.pkl')
save_model(optimized_ad, 'Model/AdBoost.pkl')
save_model(hgb, 'Model/HistogramGradientBoost.pkl')



In [38]:

optimized_gb = pickle.load(open('Model/GradientBoost.pkl', 'rb'))

In [39]:
optimized_rf = pickle.load(open('Model/RandomForest.pkl', 'rb'))

In [44]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

In [45]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(32, kernel_initializer='normal',input_dim = train_X.shape[1], activation='relu'))

# The Hidden Layers :
#NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
#NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
#NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                608       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 44,193
Trainable params: 44,193
Non-trainable params: 0


In [48]:
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=5)

#compile model using mse as a measure of model performance
NN_model.compile(optimizer='adam', loss='mean_squared_error')
#train model
NN_model.fit(train_X, train_Y, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

# save model and architecture to single file
NN_model.save("Model/NNmodel_weights.h5")
print("Saved model to disk")

Train on 14028 samples, validate on 3508 samples
Epoch 1/30
14028/14028 [==============================] - 2s 149us/step - loss: 1336.5200 - val_loss: 1421.2619
Epoch 2/30
14028/14028 [==============================] - 1s 100us/step - loss: 1339.7453 - val_loss: 1384.4057
Epoch 3/30
14028/14028 [==============================] - 1s 102us/step - loss: 1324.1006 - val_loss: 1361.8247
Epoch 4/30
14028/14028 [==============================] - 1s 96us/step - loss: 1318.5148 - val_loss: 1373.1921
Epoch 5/30
14028/14028 [==============================] - 1s 97us/step - loss: 1325.4880 - val_loss: 1375.8630
Epoch 6/30
14028/14028 [==============================] - 1s 102us/step - loss: 1324.5616 - val_loss: 1358.3411
Epoch 7/30
14028/14028 [==============================] - 1s 103us/step - loss: 1326.9310 - val_loss: 1357.3564
Epoch 8/30
14028/14028 [==============================] - 1s 93us/step - loss: 1335.3459 - val_loss: 1475.7237
Epoch 9/30
14028/14028 [==============================] - 

In [51]:
wights_file = 'Model/NNmodel_weights.h5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [52]:
nnmodel_pred = NN_model.predict(test_X)
model_perf_nn = createOutParam('NN_Model', test_X, test_Y, nnmodel_pred)

In [57]:
model_perf.append(model_perf_nn)
Model_Compare = pd.DataFrame(model_perf)
Model_Compare.drop_duplicates(subset=None, keep='first', inplace=True)

In [58]:
Model_Compare.style.background_gradient(cmap='Blues')

,ModelName,R_squared,explained_variance,max_error,mean_absolute_error,mean_squared_error,median_absolute_error,root_mean_squared_error
0,RandomForestRegressor,0.725298,0.725316,156.998,25.2857,1286.81,17.4999,35.8721
1,ElasticNet,0.648486,0.648487,175.465,31.1589,1646.63,25.9823,40.5786
2,SVM_Regressor,0.679144,0.688798,182.324,26.8343,1503.01,17.6821,38.7687
3,GradientBoostingRegressor,0.720121,0.720137,161.785,25.6118,1311.06,17.3578,36.2086
4,VotingRegressor,0.713834,0.714342,168.442,26.0709,1340.51,18.521,36.613
5,AdaBoostRegressor,0.678562,0.682231,153.791,29.2608,1505.74,21.9434,38.8038
6,HistGradientBoostingRegressor,0.719844,0.719845,160.998,25.6477,1312.36,17.6453,36.2265
7,XGBoost,0.721311,0.721312,163.441,25.6129,1305.49,17.7259,36.1315
8,Stacking_Regressor,0.725895,0.725902,160.989,25.2591,1284.01,17.2931,35.8331
9,NN_Model,0.716959,0.724264,153.694,26.5151,1325.87,19.5354,36.4125


### Conclusion

* ElastiNet Regressor is the worst performing Model
* RandomForest Regressor and Stacking Regressor are the Best performing Models
* Stacking Regressor has been chosen as the final Model
